# Interakcija SSH/Telnet

Viri:
- [Welcome to Paramiko!](http://www.paramiko.org/)
- [SSH & SCP in Python with Paramiko](https://hackersandslackers.com/automate-ssh-scp-python-paramiko/)
- http://www.fabfile.org/ -> dodaj

## SSH

Cloud providers have made a killing from neatly-packaged managed services for years. Whether it be databases or message brokers, developers like ourselves don't seem to have a problem paying a bit extra to have things taken care of. But wait, aren't we typically the last people to opt for less optimization and less control? Why is this the time we decide otherwise? If I had to make a guess, I'd wager it's because server-side DevOps kind of sucks.

As a developer, configuring or debugging a VPS is usually work which is unaccounted for, and it isn't particularly rewarding. At best, your application will probably end up running the same as your local environment. How could we make this inevitable part of our jobs better? Well, we could automate  it.

paramiko and scp are two Python libraries we can use together to automate tasks we'd want to run on a remote host such as restarting services, making updates, or grabbing log files. We're going to take a look at what scripting with these libraries looks like. Fair warning: there's a sizable amount of code in this tutorial, which tends to make me excited enough to coerce others into my manic code-tutorial episodes. If you ever begin to feel lost, the full repo can be found here: https://github.com/hackersandslackers/paramiko-tutorial/tree/master/paramiko_tutorial

> Paramiko is a Python (2.7, 3.4+) implementation of the SSHv2 protocol, providing both client and server functionality. While it leverages a Python C extension for low level cryptography (Cryptography), Paramiko itself is a pure Python interface around SSH networking concepts.

> [Pure python scp module](https://github.com/jbardin/scp.py) he scp.py module uses a paramiko transport to send and recieve files via the scp1 protocol. This is the protocol as referenced from the openssh scp program, and has only been tested with this implementation.

### Installation

Let's install our libraries. Fire up whichever virtual environment you prefer and let em rip:

    pip3 install paramiko scp

### Making a connection

The primary client of Paramiko as documented in the API, is Paramiko.SSHClient. An instance of the Paramiko.SSHClient can be used to make connections to the remote server and transfer files.

In [2]:
import paramiko

ssh_client=paramiko.SSHClient()
ssh_client.connect(hostname='212.101.137.47', port=22, username='sshtest', password='EnoVelikoDrevoCesenj12!')
#paramiko.ssh_exception.SSHException: Server '[192.168.56.101]:2022' not found in known_hosts 

SSHException: Server '212.101.137.47' not found in known_hosts

Connect to an SSH server and authenticate to it. The server’s host key is checked against the system host keys (see load_system_host_keys) and any local host keys (load_host_keys). If the server’s hostname is not found in either set of host keys, the missing host key policy is used (see set_missing_host_key_policy). The default policy is to reject the key and raise an SSHException.

Authentication is attempted in the following order of priority:
- The pkey or key_filename passed in (if any)
    - key_filename may contain OpenSSH public certificate paths as well as regular private-key paths; when files ending in -cert.pub are found, they are assumed to match a private key, and both components will be loaded. (The private key itself does not need to be listed in key_filename for this to occur - just the certificate.)
- Any key we can find through an SSH agent
- Any “id_rsa”, “id_dsa” or “id_ecdsa” key discoverable in ~/.ssh/
    - When OpenSSH-style public certificates exist that match an existing such private key (so e.g. one has id_rsa and id_rsa-cert.pub) the certificate will be loaded alongside the private key and used for authentication.
- Plain username/password auth, if a password was given

If a private key requires a password to unlock it, and a password is passed in, that password will be used to attempt to unlock the key.

When you try this, you get the following error:
missing_host_key raise SSHException(‘Server %r not found in known_hosts’ % hostname) paramiko.ssh_exception.SSHException: Server ‘hostname’ not found in known_hosts

You see this error because you have not informed your machine that the remote server you “trust” the server you are trying to access. If you go onto you command line or terminal and try to connect to a server for the first time, You will get a message similar to this:

The authenticity of host ‘hostname’ can’t be established.RSA key fingerprint is ‘key’. Are you sure you want to continue connecting (yes/no)?

When you select yes here, you let your machine know that it can trust the machine and you can now access it without the prompt until the key for that machine changes.
Paramiko similarly requires that you validate your trust with the machine. This validation is handled by calling set_missing_host_key_policy() on the SSHClient an passing the policy you want implemented when accessing a new remote machine. By default, the paramiko.SSHclient sets the policy to the RejectPolicy. The policy rejects connection without validating as we saw above. Paramiko does however give you a way to sort of “Trust all” key policy, the AutoAddPolicy. Parsing an instance of the AutoAddPolicy to set_missing_host_key_policy() changes it to allow any host.

> Obligatory warning: Do not use AutoAddPolicy, unless you do not care about security. You are losing a protection against MITM attacks this way. more info: https://stackoverflow.com/questions/10670217/paramiko-unknown-server#43093883

In [3]:
import paramiko
ssh_client =paramiko.SSHClient()
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
# bolj varna opcija - https://docs.paramiko.org/en/stable/api/client.html#paramiko.client.SSHClient.get_host_keys
#ssh_client.load_system_host_keys()
ssh_client.connect(hostname='212.101.137.47', port=22, username='sshtest', password='EnoVelikoDrevoCesenj12!')
# neki delamo 
print('Working')
ssh_client.close()

Working


> `close() - Close this SSHClient and its underlying Transport.`: Failure to do this may, in some situations, cause your Python interpreter to hang at shutdown (often due to race conditions). It’s good practice to close your client objects anytime you’re done using them, instead of relying on garbage collection.

#### Setting up SSH Keys

> Na napravi na kateri izvajamo skripte

To authenticate an SSH connection, we need to set up a private RSA SSH key (not to be confused with OpenSSH). We can generate a key using the following command:

    ssh-keygen -t rsa

This will prompt us to provide a name for our key. Name it whatever you like:

Next, you'll be prompted to provide a password (feel free to leave this blank). -> plume

Now that we have our key, we need to copy this to our remote host. The easiest way to do this is by using ssh-copy-id:

    ssh-copy-id -i ~/.ssh/id_rsa.pub testuser@ssh-telnet-test

> preverimo ali povezava deluje

In [6]:
import paramiko
ssh_client =paramiko.SSHClient()
#ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
# bolj varna opcija - https://docs.paramiko.org/en/stable/api/client.html#paramiko.client.SSHClient.get_host_keys
ssh_client.load_system_host_keys()
ssh_client.connect(hostname='212.101.137.47', port=22, username='sshtest', passphrase='leons')
# neki delamo 
print('Working')
ssh_client.close()

Working


### Running commands on the remote machine

> `exec_command(command, bufsize=-1, timeout=None, get_pty=False, environment=None)` Execute a command on the SSH server. A new Channel is opened and the requested command is executed. The command’s input and output streams are returned as Python file-like objects representing stdin, stdout, and stderr.

**Parameters**:
- command (str) – the command to execute
- bufsize (int) – interpreted the same way as by the built-in file() function in Python
- timeout (int) – set command’s channel timeout. See Channel.settimeout
- get_pty (bool) – Request a pseudo-terminal from the server (default False). See Channel.get_pty
- environment (dict) – a dict of shell environment variables, to be merged into the default environment that the remote command executes within.

**Returns**:
- the stdin, stdout, and stderr of the executing command, as a 3-tuple

**Raises**:
- SSHException – if the server fails to execute the command



To run a command exec_command is called on the SSHClient with the command passed. The response is returned as a tuple (stdin,stdout,stderr)
For example to list all the files in a directory:

    stdin,stdout,stderr=ssh_client.exec_command(“ls”)

Getting the type for each of the returned,
type(stdin) and type(stdout) is ‘paramiko.channel.ChannelFile’
type(stderr) is class ‘paramiko.channel.ChannelStderrFile’

According to paramiko.org they are all python file like objects.

The stdin is a write-only file which can be used for commands requiring input
The stdout file give the output of the command
The stderr gives the errors returned on executing the command. Will be empty if there is no error

    # branje vrednosti
    print(stdout.readlines())
    print(stderr.readlines())

> Return the exit status from the process on the server. This is mostly useful for retrieving the results of an exec_command. If the command hasn’t finished yet, this method will wait until it does, or until the channel is closed. If no exit status is provided by the server, -1 is returned.

In [ ]:
import paramiko

def execute_command(client, command):
    """Execute command in succession."""
    stdin, stdout, stderr = client.exec_command(command)
    status = stdout.channel.recv_exit_status()
    if status != 0:
        print(f'Status: {status}, message: {stderr.readlines()}')
    response = stdout.readlines()
    client.close()
    print(response)

ssh_client = paramiko.SSHClient()
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh_client.connect(hostname='ssh-telnet-test', port=22, username='testuser', password='test')
command = "uname -a"

execute_command(ssh_client, command)

#### COMMANDS REQUIRING INPUT

Sometimes you need to provide a password or extra input to run a command. This is what stdin is used for. Let’s run the same command above with sudo.

In [11]:
ssh_client.load_system_host_keys()
ssh_client.connect(hostname='212.101.137.47', port=22, username='sshtest', passphrase='leons')

stdin, stdout, stderr = ssh_client.exec_command("sudo ls -la")
stdin.write("EnoVelikoDrevoCesenj12!")
stdin.write("\n")
stdin.flush()
print(stdout.readlines())
print(stderr.readlines())

print('Working')
ssh_client.close()

[]
['sudo: a terminal is required to read the password; either use the -S option to read from standard input or configure an askpass helper\n']
Working


Should return list of files and folders as above.

## Primer: Create a SSH/SCP program

### Starting our Script

Just one more thing before we write some meaningful Python code! Create a config file to hold the variables we'll need to connect to our host. Here are the barebones of what we need to get into our server:
- Host: The IP address or URL of the remote host we're trying to access.
- Username: This is the username you use to SSH into your server.
- Passphrase (optional): If you specified a passphrase when you created your ssh key, specify that here. Remember that your SSH key passphrase is not the same as your user's password.
- SSH Key: The file path of the key we created earlier. On OSX, these live in your system's ~/.ssh folder. SSH key we're targeting must have an accompanying key with a .pub file extension. This is our public key; if you were following along earlier, this should have already been generated for you.

If you're trying to upload or download files from your remote host, you'll need to include two more variables:
- Remote Path: The path to the remote directory we're looking to target for file transfers. We can either upload things to this folder or download the contents of it.
- Local Path: Same idea as above, but the reverse. For our convenience, the local path we'll be using is simply /data, and contains pictures of cute fox gifs.

Now we have everything we need to make a respectable config.py file:

In [ ]:
# config.py
"""Configuration module."""
from os import environ

host = environ.get('REMOTE_HOST')
user = environ.get('REMOTE_USERNAME')
password = environ.get('REMOTE_PASSWORD')

remote_path = environ.get('REMOTE_PATH')
local_path = '/home/plume/data'

### Creating an SSH Client

We're going to create a class called RemoteClient to handle the interactions we'll be having with our remote host. Before we get too fancy, let's just start things off by instantiating the RemoteClient class with the variables we created in config.py:

In [ ]:
# client.py
class RemoteClient:
    """Client to interact with a remote host via SSH & SCP."""

    def __init__(self, host, user, password, remote_path):
        self.host = host
        self.user = user
        self.password = password
        self.remote_path = remote_path
        self.client = None
        self.scp = None

Nothing impressive so far: we've just set some variables and passed them into a useless class. Let's kick things up a notch without leaving our constructor:

In [ ]:
# client.py
"""Remote host object to handle connections and actions."""
from paramiko import SSHClient, AutoAddPolicy, RSAKey
from paramiko.auth_handler import AuthenticationException, SSHException


class RemoteClient:
    """Client to interact with a remote host via SSH & SCP."""

    def __init__(self, host, user, password, remote_path):
        self.host = host
        self.user = user
        self.password = password
        self.remote_path = remote_path
        self.client = None
        self.scp = None


We've added three new things to be instantiated with our class:
- self.client = None: self.client will ultimately serve as the connection objection in our class, similar to how you have dealt with terminology like conn in database libraries. Our connection will be None until we explicitly connect to our remote host.
- self.scp = None Similar to self.client, but exclusively handles connections for transferring files.
- `self.__upload_ssh_key()` isn't a variable, but rather a function to be run automatically whenever our client is instantiated. Calling `__upload_ssh_key()` is telling our RemoteClient object to check for local ssh keys immediately upon creation so that we can try to pass them to our remote host. Otherwise, we wouldn't be able to establish a connection at all.

### Uploading SSH Keys to a Remote Host

We've reached the section of this exercise where we need to knock out some devastatingly inglorious boilerplate code. This is typically where emotionally inferior individuals succumb to the sheer dull obscurity of understanding SSH keys and maintaining connections. Make no mistake: authenticating and managing connections to anything programmatically is overwhelmingly dull... unless your tour guide happens to be an enchanting wordsmith, serving as your loving protector through perilous obscurity. Some people call this post a tutorial. I intend to call it art.

RemoteClient will start with two private methods: __get_ssh_key()  and __upload_ssh_key(). The former will fetch a locally stored public key, and if successful, the latter will deliver this public key to our remote host as an olive branch of access. Once a locally created public key exists on a remote machine, that machine will then forever trust us our requests to connect to it: no passwords required. We'll be including proper logging along the way, just in case we run into any trouble:

In [ ]:
"""Remote host object to handle connections and actions."""
import sys
from loguru import logger
from os import system
from paramiko import SSHClient, AutoAddPolicy
from scp import SCPClient, SCPException


logger.add(sys.stderr,
           format="{time} {message}",
           filter="client",
           level="INFO")
logger.add('logs/log_{time:YYYY-MM-DD}.log',
           format="{time} {level} {message}",
           filter="client",
           level="ERROR")


class RemoteClient:
    """Client to interact with a remote host via SSH & SCP."""
    def __init__(self, host, user, password=None, passphrase=None, remote_path='~/', port=22):
        self.host = host
        self.user = user
        self.password = password
        self.passphrase = passphrase
        self.remote_path = remote_path
        self.port = port
        self.client = None
        self.scp = None

__get_ssh_key() is quite simple: it verifies that an SSH key exists at the path we specified in our config to be used for connecting to our host. If the file does in fact exist, we happily set our self.ssh_key variable, so this key can be uploaded and used by our client from here forward. Paramiko provides us with a submodule called RSAKey to easily handle all things RSA key related, like parsing a private key file into a usable connection authentication. That's what we get here:

    RSAKey.from_private_key_file(self.ssh_key_filepath)

If our RSA key were incomprehensible nonsense instead of a real key, Paramiko's SSHException would have caught this and raised an exception early on explaining just that. Properly utilizing a library's error handling takes a lot of the guesswork out of "what went wrong," especially in cases like where there's potential for numerous unknowns in a niche space neither of us mess with often.

__upload_ssh_key() is where we get to jam our SSH key down the throat of our remote server while shouting, "LOOK! YOU CAN TRUST ME FOREVER NOW!" To accomplish this, I go a bit "old school" by passing bash commands via Python's os.system. Unless somebody makes me aware of a cleaner approach in the comments, I'll assume this is the most badass way to handle passing keys to a remote server.

The standard non-Python way of passing keys to a host looks like this:

    ssh-copy-id -i ~/.ssh/mykey user@host

This is precisely what we accomplish in our function in Python, which looks like this:

    system(f'ssh-copy-id -i {self.ssh_key_filepath} {self.user}@{self.host}>/dev/null 2>&1')

I suppose you won't let me slip that /dev/null 2>&1 bit by you? Fine. If you must know, here's some guy on StackOverflow explaining it better than I can:

> \> is for redirect /dev/null is a black hole where any data sent, will be discarded. 2 is the file descriptor for Standard Error. > is for redirect. & is the symbol for file descriptor (without it, the following 1 would be considered a filename). 1 is the file descriptor for Standard O.

So we're basically telling our remote server we're giving it something, and it's all like "where do I put this thing," to which we reply "nowhere in physical in space, as this is not an object, but rather an eternal symbol of our friendship.  Our remote host is then flooded with gratitude and emotion, because yes, computers do have emotions, but we can't be bothered by that right now.

### Connecting to our Client

We'll add a method to our client called connect() to handle connecting to our host:

In [ ]:
class RemoteClient:
    """Client to interact with a remote host via SSH & SCP."""

    #...
    def __connect(self):
        """Open connection to remote host."""
        self.client = SSHClient()
        self.client.set_missing_host_key_policy(AutoAddPolicy())
        self.client.load_system_host_keys()
        self.client.connect(self.host,
                            username=self.user,
                            timeout=5000,
                            port=self.port,
                            passphrase=self.passphrase,
                            password=self.password
                            )
        self.scp = SCPClient(self.client.get_transport())
        return self.client

Let's break this down:
- client = SSHClient() sets the stage for creating an object representing our SSH client. The following lines will configure this object to make it more useful.
- load_system_host_keys() instructs our client to look for all the hosts we've connected to in the past by looking at our system's known_hosts file and finding the SSH keys our host is expecting. We've never connected to our host in the past, so we need to specify our SSH key explicitly.
- set_missing_host_key_policy() tells Paramiko what to do in the event of an unknown key pair. This is expecting a "policy" built-in to Paramiko, to which we're going to specific AutoAddPolicy(). Setting our policy to "auto-add" means that if we attempt to connect to an unrecognized host, Paramiko will automatically add the missing key locally.
- connect() is SSHClient's most important method (as you might imagine). We're finally able to pass our host, user, and SSH key to achieve what we've all been waiting for: a glorious SSH connection into our server! The connect() method allows for a ton of flexibility via a vast array of optional keyword arguments too. I happen to pass a few here: setting look_for_keys to True gives Paramiko permission to look around in our ~/.ssh folder to discover SSH keys on its own, and setting timeout will automatically close connections we'll probably forget to close. We could even pass variables for things like port and password, if we had elected to connect to our host this way.

### Disconnecting

We should close connections to our remote host whenever we're done using them. Failing to do so might not necessarily be disastrous, but I've had a few instances where enough hanging connections would eventually max out inbound traffic on port 22. Regardless of whether your use case might consider a reboot to be a disaster or mild inconvenience, let's just close our damn connections like adults as though we were wiping our butts after pooping. No matter your connection hygiene, I advocate for setting a timeout variable (as we saw earlier). Anyway. voila:

In [ ]:
class RemoteClient:
    #...

    def disconnect(self):
        """Close ssh connection."""
        self.client.close()
        self.scp.close()


Fun fact: setting self.client.close() actually sets self.client to equal None, which is useful in cases where you might want to check if a connection is already open.

### Executing Unix Commands

We now have a wonderful Python class that can find RSA keys, connect, and disconnect. It does lack the ability to do, well, anything useful.

We can fix this and finally begin doing "stuff" with a brand new method to execute commands, which I'll aptly dub execute_commands() (that's correct, "commands" as in potentially-more-than-one, we'll touch on that in a moment). The legwork of all this is done by the Paramiko client's built-in exec_command() method, which accepts a single string as a command and executes it:

In [ ]:
class RemoteClient:
    #...

    def execute_commands(self, commands):
        """Execute multiple commands in succession."""
        if self.client is None:
            self.client = self.__connect()
        for cmd in commands:
            stdin, stdout, stderr = self.client.exec_command(cmd)
            stdout.channel.recv_exit_status()
            response = stdout.readlines()
            for line in response:
                logger.info(f'INPUT: {cmd} | OUTPUT: {line}')

The function we just created execute_commands() expects a list of strings to execute as commands. That's partially for convenience, but it's also because Paramiko won't run any "state" changes (like changing directories) between commands, so each command we pass to Paramiko should assume we're working out of our server's root. I took the liberty of passing three such commands like so:

    remote.execute_commands(['cd /var/www/ && ls',
                            'tail /var/log/nginx/access.log',
                            'ps aux | grep node'])

I can view the contents of a directory by chaining cd path/to/dir && ls, but running cd path/to/dir followed by ls would result in nothingness because ls the second time returns the list of files in our server's root.

You'll notice client.exec_command(cmd) returns three values as opposed to one: this can be useful to see which input produced which output. For example, here are the full logs for the example I provided where I passed three commands to remote.execute_commands():

Some beautiful stuff here. Now you can see which sites are on my server, which bots are spamming me, and how many node processes I'm running.

I don't want to waste much more time on the art of executing commands, but it's worth mentioning the presence of why we call stdout.channel.recv_exit_status() after each command. Waiting for recv_exit_status() to come back after running client.exec_command() forces our commands to be run synchronously, otherwise there's a likely chance our remote machine won't be about to decipher commands as fast as we pass them along.

### Uploading (and Downloading) Files via SCP

SCP refers to both the protocol for copying files to remote machines (secure copy protocol) as well as the Python library, which utilizes this. We've already installed the SCP library, so import that shit.

The SCP and Paramiko libraries complement one another to make uploading via SCP super easy. SCPClient() creates an object which expects  "transport" from Paramiko, which we provide with self.conn.get_transport(). Creating an SCP connection piggybacks off of our SSH client in terms of syntax, but these connections are separate. It's possible to close an SSH connection and leave an SCP connection open, so don't do that. Open an SCP connection like this:

    self.scp = SCPClient(self.client.get_transport())

Uploading a single file is boring, so let's upload an entire directory of files instead. bulk_upload() accepts a list of file paths, and then calls __upload_single_file()

In [ ]:
class RemoteClient:
    #...

    def bulk_upload(self, files):
        """Upload multiple files to a remote directory."""
        if self.client is None:
            self.client = self.__connect()
        uploads = [self.__upload_single_file(file) for file in files]
        logger.info(f'Finished uploading {len(uploads)} files to {self.remote_path} on {self.host}')

    def __upload_single_file(self, file):
        """Upload a single file to a remote directory."""
        try:
            self.scp.put(file,
                         recursive=True,
                         remote_path=self.remote_path)
        except SCPException as error:
            logger.error(error)
            raise error
        finally:
            logger.info(f'Uploaded {file} to {self.remote_path}')

Our method is expecting to receive two strings: the first being the local path to our file, and the second being the path of the remote directory we'd like to upload to.

SCP's put() method will upload a local file to our remote host. This will replace existing files with the same name if they happen to exist at the destination we specify. That's all it takes!

The counterpart to SCP's put() is the get() method:

In [ ]:
class RemoteClient:

    #...

    def download_file(self, file):
        """Download file from remote host."""
        if self.client is None:
            self.client = self.__connect()
        self.scp.get(file)

### Run the script

We now have a sick Python class to handle SSH and SCP with a remote host... let's put it to work! The following snippet is a quick way to test what we've built so far. In short, this script looks for a local folder filled with files (in my case, I filled the folder with fox gifs 🦊).

Check out how easy it is to create a main.py that handles complex tasks on remote machines thanks to our RemoteClient class:

> npr.:  `export REMOTE_HOST=ssh-telnet-test`

In [ ]:
"""Perform tasks against a remote host."""
from client import RemoteClient
from files import fetch_local_files

host = 'ssh-telnet-test'
port = 22
# credentials
user='testuser'
password='test'
passphrase='plume'
# file upload
local_file_directory='/home/plume/vsebina/osnovni_tecaj/11_Interakcija_SSH_Telnet/skripte/ssh_scp_module/data'
remote_path='/home/testuser/data'

def remote = RemoteClient(host, user, passphrase=passphrase, password=password, remote_path=remote_path, port=22)():
    """Initialize remote host client and execute actions."""
    remote = RemoteClient(host, user, passphrase=passphrase, password=password, remote_path=remote_path, port=22)
    #execute_command_on_remote(remote)
    download_file_from_remote(remote)
    #upload_files_to_remote(remote)
    remote.disconnect()
    
def upload_files_to_remote(remote):
    """Upload files to remote via SCP."""
    files = fetch_local_files(local_file_directory)
    remote.bulk_upload(files)

def download_file_from_remote(remote):
    remote.download_file('/home/testuser/some.txt')


def execute_command_on_remote(remote):
    """Execute UNIX command on the remote host."""
    remote.execute_commands(['uname -a'])

main()

## Telnet

The [telnetlib module](https://docs.python.org/3/library/telnetlib.html) provides a Telnet class that implements the Telnet protocol. See RFC 854 for details about the protocol. In addition, it provides symbolic constants for the protocol characters (see below), and for the telnet options.

`class telnetlib.Telnet(host=None, port=0[, timeout])`

Telnet represents a connection to a Telnet server. The instance is initially not connected by default; the open() method must be used to establish a connection. Alternatively, the host name and optional port number can be passed to the constructor too, in which case the connection to the server will be established before the constructor returns. The optional timeout parameter specifies a timeout in seconds for blocking operations like the connection attempt (if not specified, the global default timeout setting will be used).

Do not reopen an already connected instance.

This class has many read_*() methods. Note that some of them raise EOFError when the end of the connection is read, because they can return an empty string for other reasons. See the individual descriptions below.

A Telnet object is a context manager and can be used in a with statement. When the with block ends, the close() method is called:

In [ ]:
from telnetlib import Telnet

with Telnet('localhost', 23) as tn:
    tn.interact()

`Telnet.read_until(expected, timeout=None)`

Read until a given byte string, expected, is encountered or until timeout seconds have passed.

When no match is found, return whatever is available instead, possibly empty bytes. Raise EOFError if the connection is closed and no cooked data is available.

`Telnet.write(buffer)`

Write a byte string to the socket, doubling any IAC characters. This can block if the connection is blocked. May raise OSError if the connection is closed.

Raises an auditing event telnetlib.Telnet.write with arguments self, buffer.

`Telnet.read_all()`

Read all data until EOF as bytes; block until connection closed.

`getpass.getpass(prompt='Password: ', stream=None)`

[getpass — Portable password input](https://docs.python.org/3.8/library/getpass.html)

Prompt the user for a password without echoing. The user is prompted using the string prompt, which defaults to 'Password: '. On Unix, the prompt is written to the file-like object stream using the replace error handler if needed. stream defaults to the controlling terminal (/dev/tty) or if that is unavailable to sys.stderr (this argument is ignored on Windows).

If echo free input is unavailable getpass() falls back to printing a warning message to stream and reading from sys.stdin and issuing a GetPassWarning.

A simple example illustrating typical use:

In [ ]:
import getpass
import telnetlib

HOST = "ssh-telnet-test"

user = 'testuser' 
#user = input("Enter your remote account: ")

password = 'test' 
#password = getpass.getpass()


with telnetlib.Telnet(HOST) as tn:
    tn.read_until(b"login: ")
    tn.write(user.encode('ascii') + b"\n")

    if password:
        tn.read_until(b"Password: ")
        tn.write(password.encode('ascii') + b"\n")

    tn.write(b"ls\n")
    tn.write(b"mkdir /home/testuser/telnet-test \n")

    tn.write(b"exit\n")

    print(tn.read_all().decode('ascii'))

### Primer uporabe: Telnet

In [ ]:
import getpass
import telnetlib

def create_dirs(dir_list, path, host, user, password, port=23):
    with telnetlib.Telnet(host, port) as tn:
        print(f"--> connected to host: {host} on port: {port}")
        
        tn.read_until(b"login: ")
        tn.write(user.encode('ascii') + b"\n")

        if password:
            tn.read_until(b"Password: ")
            tn.write(password.encode('ascii') + b"\n")
        
        print(f"--> succsessful login for user: {user}")
            
        for dir_name in dir_list:
            path_name = f'{path}/{dir_name}'
            command = f'mkdir -v {path_name}\n'
            tn.write(command.encode('ascii'))
            succsess_mesage = f"mkdir: created directory '{path_name}'".encode('ascii')
            res = tn.read_until(succsess_mesage, timeout=2)
            if succsess_mesage in res:
                print(f'--> {dir_name} created successfully')
            else:
                print(f'--> {dir_name} creation failed')

        tn.write(b"exit\n")
        print(tn.read_all().decode('ascii'))


create_dirs(['dir1', 'dir2', 'dir3'], path='/home/testuser/telnet-test', 
                    host='ssh-telnet-test',
                    user='testuser',
                    password='test')